In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!pip install wavespectra[extra]

In [ ]:
!pip install tqdm
import wavespectra
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.colorbar as cb
from matplotlib.colors import Colormap
import numpy as np
from PIL import Image
from tqdm import tqdm

## Read data from .netdcf files

In [ ]:
speccyO = wavespectra.read_netcdf('/content/gdrive/MyDrive/WaveSpectra_CAE/data/BoundarySpectrum.netcdf')
speccyI = wavespectra.read_netcdf('/content/gdrive/MyDrive/WaveSpectra_CAE/data/NearshoreSpectrum_GC.netcdf')

Check for and remove any duplicates

In [ ]:
speccyO = speccyO.sel(time=~speccyO.indexes['time'].duplicated())
speccyI = speccyI.sel(time=~speccyI.indexes['time'].duplicated())
speccyO = speccyO.sel(time=slice('2001-01-01', '2020-01-01'))
speccyI = speccyI.sel(time=slice('2001-01-01', '2020-01-01'))
speccyO = speccyO.isel(dict(lat = 0, lon = 0))

Preview the offshore/near shore spectra.

In [ ]:
speccyO.efth.sortby('dir').isel(time=slice(80000, 80002)).plot(x= 'freq', y = 'dir', col = 'time', col_wrap = 4)
speccyI.efth.sortby('dir').isel(time=slice(80000, 80002)).plot(x= 'freq', y = 'dir', col = 'time', col_wrap = 4)

###Save data as compressed .npz files

* Inidividual samples are numbered according to their position in the xarray, starting from '2001-01-01'
* Manually create destination folders (data/Offshore and data/Nearshore); change paths accordingly

In [ ]:
num_samples = 8760 * 2  # 8760 samples per year

offshore_data = [None] * num_samples
offshore_name = [None] * num_samples
nearshore_data = [None] * num_samples
nearshore_name = [None] * num_samples

for i in tqdm(range(0, num_samples)):
  offshore_data[i] = np.array(speccyO.efth.sortby('dir').isel(time=i))
  offshore_name[i] = str(i).zfill(5)
  nearshore_data[i] = np.array(speccyI.efth.sortby('dir').isel(time=i))
  nearshore_name[i] = str(i).zfill(5)

offshore_dict = dict(zip(offshore_name, offshore_data))
np.savez_compressed('/content/gdrive/MyDrive/WaveSpectra_CAE/data/offshore_2yr', **offshore_dict)
print("Offshore data saved\n")

nearshore_dict = dict(zip(nearshore_name, nearshore_data))
np.savez_compressed('/content/gdrive/MyDrive/WaveSpectra_CAE/data/nearshore_2yr', **nearshore_dict)
print("Near shore data saved\n")

In [ ]:
# Check the .npy files are being saved correctly
# Change paths accordingly

sample_index = 7564

offshore = np.load('/content/gdrive/MyDrive/WaveSpectra_CAE/data/offshore.npz')
offshore.files
plt.imshow(offshore[str(sample_index).zfill(5)].astype(np.float32), cmap='gray')
plt.axis('off')
plt.pad_inches = 0
plt.tight_layout
plt.show()

nearshore = np.load('/content/gdrive/MyDrive/WaveSpectra_CAE/data/nearshore.npz')
nearshore.files
plt.imshow(nearshore[str(sample_index).zfill(5)].astype(np.float32), cmap='gray')
plt.axis('off')
plt.pad_inches = 0
plt.tight_layout
plt.show()
